In [1]:
import numpy as np
from scipy.integrate import solve_ivp

class LQR:
    def __init__(self, A, B, Q, R, x0, t_span):
        self.A = A
        self.B = B
        self.Q = Q
        self.R = R
        self.x0 = x0
        self.t_span = t_span
        
        self.P = np.zeros_like(Q)
        self.K = np.zeros((B.shape[1], A.shape[0]))
        
    def riccati_solver(self, t, P):
        P = P.reshape(self.Q.shape)
        dPdt = -self.Q - self.A.T @ P - P @ self.A + P @ self.B @ np.linalg.inv(self.R) @ self.B.T @ P
        return dPdt.reshape(-1)
    
    def compute_gain_matrix(self):
        sol = solve_ivp(self.riccati_solver, self.t_span, self.P.reshape(-1))
        self.P = sol.y[:, -1].reshape(self.Q.shape)
        self.K = np.linalg.inv(self.R) @ self.B.T @ self.P
        
    def compute_control_input(self, x):
        return -self.K @ x
    
    def simulate(self):
        sol = solve_ivp(lambda t, x: self.A @ x + self.B @ self.compute_control_input(x), self.t_span, self.x0)
        return sol.t, sol.y


In [4]:
A = np.array([[0, 1], [-1, 0]])
B = np.array([[0], [1]])
Q = np.eye(2)
R = np.array([[1]])
x0 = np.array([1, 0])
t_span = (0, 10)

lqr = LQR(A, B, Q, R, x0, t_span)



In [6]:
lqr.compute_gain_matrix()

t, x = lqr.simulate()
 


array([[ 1.00000000e+00,  9.99999647e-01,  9.99957158e-01,
         9.95498259e-01,  6.83578731e-01, -1.22157946e+00,
        -5.14900650e+00, -8.76572393e+00, -8.27704518e-02,
         3.22880784e+01,  7.21567130e+01,  3.83611519e+01,
        -1.94879595e+02, -5.57266262e+02, -6.00144152e+02],
       [ 0.00000000e+00, -9.99478165e-04, -1.10467895e-02,
        -1.16811840e-01, -1.18252787e+00, -3.64266990e+00,
        -5.34902144e+00, -2.86882614e-01,  2.02175595e+01,
         4.47364897e+01,  2.74176521e+01, -1.18840985e+02,
        -3.52420056e+02, -3.50110534e+02,  4.34675998e+02]])